## ETL Project: TrueSafeCars.com
Group 8 (NoFloods)
* Roopa Patel
* Samuel Parks
* Steven Lee


In [38]:
import pandas as pd
import os
import numpy as np
import datetime
import requests
import json

### EXTRACT Datasets

In [2]:
# Define our input datasets and load into dataframes
# NOTE: Cars1 refers to a dataset of cars that were scraped from TrueCar.com
#       Cars2 refers to a dataset of cars that were scraped from Craigslist

# Paths to our datasets
input_file_truecar = os.path.join(".", "Datasets", "true_car_listings.csv")
input_file_craigslist = os.path.join(".", "Datasets", "vehicles.csv")

# Read files into dataframes
truecar_df = pd.read_csv(input_file_truecar)
craigslist_df = pd.read_csv(input_file_craigslist)

In [3]:
# Output Preliminary statistics

# Row counts and unique VINs
print(f"TrueCar dataframe beginning number of rows: {truecar_df.shape[0]}")
total_true_car_VINs = len(truecar_df["Vin"].unique())
print(f"TrueCar total unique VIN records: {total_true_car_VINs}\n")

print(f"Craigslist dataframe beginning number of rows: {craigslist_df.shape[0]}")
total_craigslist_VINs = len(craigslist_df["vin"].unique())
print(f"Craigslist dataframe total unique VIN records: {total_craigslist_VINs}")

TrueCar dataframe beginning number of rows: 852122
TrueCar total unique VIN records: 852075

Craigslist dataframe beginning number of rows: 539759
Craigslist dataframe total unique VIN records: 181678


In [4]:
# Counts by car make - TrueCar
truecar_grouped_by_make = truecar_df.groupby(['Make'])
truecar_count_by_make = truecar_grouped_by_make["Vin"].count()
truecar_count_by_make

Make
AM                   19
Acura             11049
Alfa                 44
Aston               149
Audi              12618
BMW               32415
Bentley             367
Buick             12491
Cadillac          15047
Chevrolet        102268
Chrysler          16357
Dodge             34368
FIAT               1782
Ferrari             345
Fisker               19
Ford             110432
Freightliner         11
GMC               29008
Genesis             141
Geo                   2
HUMMER              949
Honda             50193
Hyundai           35837
INFINITI          12258
Isuzu                76
Jaguar             2200
Jeep              40373
Kia               28636
Lamborghini         121
Land               4096
Lexus             20641
Lincoln            7120
Lotus                42
MINI               4375
Maserati           1047
Maybach              10
Mazda             13365
McLaren              47
Mercedes-Benz     26323
Mercury            1076
Mitsubishi         4080
Nissan     

In [5]:
# Counts by car make - Craigslist
craigslist_grouped_by_make = craigslist_df.groupby(['manufacturer'])
craigslist_count_by_make = craigslist_grouped_by_make["vin"].count()
craigslist_count_by_make

manufacturer
acura               2371
alfa-romeo            68
aston-martin          20
audi                3915
bmw                 9135
buick               4505
cadillac            4737
chevrolet          45185
chrysler            4760
datsun                25
dodge              10599
ferrari               45
fiat                 645
ford               58361
gmc                14481
harley-davidson      152
hennessey              0
honda              14042
hyundai             8360
infiniti            2755
jaguar               642
jeep               15592
kia                 6323
land rover             5
lexus               3908
lincoln             2009
mazda               3785
mercedes-benz       7405
mercury              799
mini                1563
mitsubishi          1597
morgan                 2
nissan             17351
pontiac             1488
porche                 6
ram                17325
rover               1346
saturn               810
subaru              7004
tesla       

### TRANSFORM Datasets

In [6]:
# Cleanup columns
#   1. Examine columns and remove unneeded columns
#   2. Rename columns to make the two datasets consistent
#   3. Reorder columns to make the two datasets consistent

# Print Before summary
print("---------------------------------")
print("BEFORE Column name cleanup:")
print("---------------------------------")
print("TrueCar dataframe columns:")
print(truecar_df.info())
print("\nCraigslist dataframe columns:")
print(craigslist_df.info())

# Drop unneeded columns
truecar_df.drop(['City'], axis=1, inplace=True)

# Drop unneeded columns
craigslist_df.drop(['id', 'url', 'region', 'region_url', 'cylinders',\
                    'fuel', 'title_status', 'transmission', 'drive',\
                    'size', 'type', 'paint_color', 'image_url',\
                    'description', 'county', 'lat', 'long'], axis=1, inplace=True)

# Add Condition column to TrueCar dataframe, used later so we can concatentate the two dataframes 
truecar_df["Condition"] = ""

# Rename Craigslist columns to be consistent with TrueCar
craigslist_df.rename(columns={"price" : "Price",
                              "year" : "Year",
                              "manufacturer" : "Make",
                              "model" : "Model",
                              "condition" : "Condition",
                              "odometer" : "Mileage",
                              "vin" : "Vin",
                              "state" : "State"}, inplace=True)

# Reorganize Craigslist columns to match cars1
craigslist_df = craigslist_df[["Price", "Year", "Mileage", "State", "Vin", "Make", "Model", "Condition"]]

# Print After summary
print("\n---------------------------------")
print("AFTER Column name cleanup:")
print("---------------------------------")
print("TrueCar dataframe columns:")
print(truecar_df.info())
print("\nCraigslist dataframe columns:")
print(craigslist_df.info())

---------------------------------
BEFORE Column name cleanup:
---------------------------------
TrueCar dataframe columns:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 852122 entries, 0 to 852121
Data columns (total 8 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   Price    852122 non-null  int64 
 1   Year     852122 non-null  int64 
 2   Mileage  852122 non-null  int64 
 3   City     852122 non-null  object
 4   State    852122 non-null  object
 5   Vin      852122 non-null  object
 6   Make     852122 non-null  object
 7   Model    852122 non-null  object
dtypes: int64(3), object(5)
memory usage: 52.0+ MB
None

Craigslist dataframe columns:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 539759 entries, 0 to 539758
Data columns (total 25 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            539759 non-null  int64  
 1   url           539759 non-null  object 
 2   region  

##### NOTE: 
* At this time, we could concatenate the two dataframes.
* However, we want to later choose random sets of rows for testing and production from each dataset.
* The main reason for this is to reduce the number of API calls.

In [7]:
# Cleanup data in columns that have text
#   1. Remove any whitespace (trim or strip) from string values
#   2. Capitalize values in these text columns consistently
#   3. Replace any NaNs in the Condition column with the text "Unknown"

# Strip any whitespace from strings
truecar_df["State"] = truecar_df["State"].str.strip()
truecar_df["Vin"] = truecar_df["Vin"].str.strip()
truecar_df["Make"] = truecar_df["Make"].str.strip()
truecar_df["Model"] = truecar_df["Model"].str.strip()
truecar_df["Condition"] = truecar_df["Condition"].str.strip()

craigslist_df["State"] = craigslist_df["State"].str.strip()
craigslist_df["Vin"] = craigslist_df["Vin"].str.strip()
craigslist_df["Make"] = craigslist_df["Make"].str.strip()
craigslist_df["Model"] = craigslist_df["Model"].str.strip()
craigslist_df["Condition"] = craigslist_df["Condition"].str.strip()

# Capitalize values in these text columns consistently
truecar_df["Make"] = truecar_df["Make"].str.title()
truecar_df["Model"] = truecar_df["Model"].str.title()
truecar_df["Condition"] = truecar_df["Condition"].str.title()
truecar_df["State"] = truecar_df["State"].str.upper()

craigslist_df["Make"] = craigslist_df["Make"].str.title()
craigslist_df["Model"] = craigslist_df["Model"].str.title()
craigslist_df["Condition"] = craigslist_df["Condition"].str.title()
craigslist_df["State"] = craigslist_df["State"].str.upper()

# Replace the NaNs in Condition with Unknown
craigslist_df.loc[craigslist_df["Condition"].isnull(),"Condition"] = "Unknown"

In [8]:
# Find problem rows for each dataset.  These are records with NaNs in any of the columns
#   1. Find rows with NaNs
#   2. If NaNs are found, extract these rows as CSVs for separate analysis by the client
#   3. Remove these rows from our dataframes

# Paths to our output files
truecar_problem_records = os.path.join(".", "ClientAnalysisNeeded", "truecar_problems.csv")
craigslist_problem_records = os.path.join(".", "ClientAnalysisNeeded", "craigslist_problems.csv")

# Find rows with NaNs in TrueCar dataset
truecar_is_NaN = truecar_df.isnull()
truecar_row_has_NaN = truecar_is_NaN.any(axis=1)
truecar_rows_with_NaN = truecar_df[truecar_row_has_NaN]

# Find rows with NaNs in Craigslist dataset
craigslist_is_NaN = craigslist_df.isnull()
craigslist_row_has_NaN = craigslist_is_NaN.any(axis=1)
craigslist_rows_with_NaN = craigslist_df[craigslist_row_has_NaN]

# If problems exist, create CSV file then remove the problem records from the dataset
print(f"Number of TrueCar problem rows found: {truecar_rows_with_NaN.shape[0]}")
if truecar_rows_with_NaN.shape[0] > 0:
    print(f"    Number of TrueCar rows BEFORE removing problem rows: {truecar_df.shape[0]}")
    truecar_rows_with_NaN.to_csv(truecar_problem_records, header=True)
    problem_index_vals = list(truecar_rows_with_NaN.index.values)
    truecar_df = truecar_df.loc[~truecar_df.index.isin(problem_index_vals), :]
    print(f"    Number of TrueCar rows AFTER removing problem rows: {truecar_df.shape[0]}")
    
print(f"Number of Craigslist problem rows found: {craigslist_rows_with_NaN.shape[0]}")    
if craigslist_rows_with_NaN.shape[0] > 0:
    print(f"    Number of Craigslist rows BEFORE removing problem rows: {craigslist_df.shape[0]}")
    craigslist_rows_with_NaN.to_csv(craigslist_problem_records, header=True)
    problem_index_vals = list(craigslist_rows_with_NaN.index.values)
    craigslist_df = craigslist_df.loc[~craigslist_df.index.isin(problem_index_vals), :]
    print(f"    Number of Craigslist rows AFTER removing problem rows: {craigslist_df.shape[0]}")


Number of TrueCar problem rows found: 0
Number of Craigslist problem rows found: 248071
    Number of Craigslist rows BEFORE removing problem rows: 539759
    Number of Craigslist rows AFTER removing problem rows: 291688


In [9]:
# Find and process duplicate VINs
#   1. Find duplicate VIN records
#   2. If duplicate VINs are found, extract these rows as CSVs for separate analysis by the client
#   3. Remove these rows from our dataframes

# Paths to our output files
truecar_dup_vins = os.path.join(".", "ClientAnalysisNeeded", "truecar_dup_vins.csv")
craigslist_dup_vins = os.path.join(".", "ClientAnalysisNeeded", "craigslist_dup_vins.csv")

# For Truecar
truecar_duplicateVinRows = truecar_df[truecar_df.duplicated(["Vin"])]
print(f"Number of TrueCar duplicate VINs found: {truecar_duplicateVinRows.shape[0]}")
if truecar_duplicateVinRows.shape[0] > 0:
    print(f"    Number of TrueCar rows BEFORE removing duplicate VINs: {truecar_df.shape[0]}")
    truecar_duplicateVinRows.to_csv(truecar_dup_vins, header=True)
    
    dup_vins_index_vals = list(truecar_duplicateVinRows.index.values)
    truecar_df = truecar_df.loc[~truecar_df.index.isin(dup_vins_index_vals), :]
    print(f"    Number of TrueCar rows AFTER removing duplicate VINs: {truecar_df.shape[0]}")

# For Craigslist
craigslist_duplicateVinRows = craigslist_df[craigslist_df.duplicated(["Vin"])]
print(f"Number of Craigslist duplicate VINs found: {craigslist_duplicateVinRows.shape[0]}")
if craigslist_duplicateVinRows.shape[0] > 0:
    print(f"    Number of Craigslist rows BEFORE removing duplicate VINs: {craigslist_df.shape[0]}")
    craigslist_duplicateVinRows.to_csv(craigslist_dup_vins, header=True)
    
    dup_vins_index_vals = list(craigslist_duplicateVinRows.index.values)
    craigslist_df = craigslist_df.loc[~craigslist_df.index.isin(dup_vins_index_vals), :]
    print(f"    Number of Craigslist rows AFTER removing duplicate VINs: {craigslist_df.shape[0]}")


Number of TrueCar duplicate VINs found: 47
    Number of TrueCar rows BEFORE removing duplicate VINs: 852122
    Number of TrueCar rows AFTER removing duplicate VINs: 852075
Number of Craigslist duplicate VINs found: 122626
    Number of Craigslist rows BEFORE removing duplicate VINs: 291688
    Number of Craigslist rows AFTER removing duplicate VINs: 169062


In [10]:
# Ensure datatypes are consistent across the two datasets
# Need to convert Craigslist "Year" and "Mileage" columns from float64 to int64

# Define conversion dictionary
conversion_dict = {"Year" : "int64",
                   "Mileage" : "int64"}

# Make the conversion
craigslist_df = craigslist_df.astype(conversion_dict)

# Confirm conversion
craigslist_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 169062 entries, 0 to 539733
Data columns (total 8 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   Price      169062 non-null  int64 
 1   Year       169062 non-null  int64 
 2   Mileage    169062 non-null  int64 
 3   State      169062 non-null  object
 4   Vin        169062 non-null  object
 5   Make       169062 non-null  object
 6   Model      169062 non-null  object
 7   Condition  169062 non-null  object
dtypes: int64(3), object(5)
memory usage: 11.6+ MB


In [11]:
# Create Sales Channel column and populate
#   - Create column based on the "Year" column
#   - If the car is >= 20 years old, then the sales channel should be "Auction",
#   - otherwise the sales channel should be "Public"

# Get current year
current_datetime = datetime.datetime.now()
current_year = current_datetime.year

# Define Public vs Auction threshold
public_vs_auction_threshold = 20

# Define function to determine Auction vs Public based on current year
def sales_or_auction(row):
    if (current_year - row["Year"] >= public_vs_auction_threshold):
        val = "Auction"
    else:
        val = "Public"
    return val

# Apply function to the TrueCar dataset
temp = truecar_df.copy()
truecar_df = temp.copy()
truecar_df["Sales_Channel"] = truecar_df.apply(sales_or_auction, axis=1)

# Apply function to the Craigslist dataset
temp = craigslist_df.copy()
craigslist_df = temp.copy()
craigslist_df["Sales_Channel"] = craigslist_df.apply(sales_or_auction, axis=1)

# Get counts by sales channel for TrueCar
truecar_sales_gb = truecar_df.groupby("Sales_Channel").count()
print("TrueCar VIN count by sales channel:")
print(truecar_sales_gb["Vin"])

# Get counts by sales channel for Craigslist
craigslist_sales_gb = craigslist_df.groupby("Sales_Channel").count()
print("\nCraigslist VIN count by sales channel:")
print(craigslist_sales_gb["Vin"])

TrueCar VIN count by sales channel:
Sales_Channel
Auction      4557
Public     847518
Name: Vin, dtype: int64

Craigslist VIN count by sales channel:
Sales_Channel
Auction      6233
Public     162829
Name: Vin, dtype: int64


In [12]:
# Update sales channel based on Condition (applies to the Craigslist dataset only)

# Define a list of Conditions that qualify for the Public sales channel
# Anything not in this list will be assigned to the Auction sales channel
public_sales_conditions = ["New", "Excellent", "Like New", "Good", "Unknown"]

craigslist_df.loc[~craigslist_df['Condition'].isin(public_sales_conditions),"Sales_Channel"] = "Auction"
craigslist_sales_gb = craigslist_df.groupby("Sales_Channel").count()
print("\nCraigslist VIN count by sales channel after updated based on condition:")
print(craigslist_sales_gb["Vin"])



Craigslist VIN count by sales channel after updated based on condition:
Sales_Channel
Auction      7360
Public     161702
Name: Vin, dtype: int64


### CREATE Test and Production Dataframes

In [13]:
# Create Test and Production dataframes by randomly selecting from our main dataframes
# Create CSV output for dataframes

# Declare output files
truecar_test_file = os.path.join(".", "TrimmedDatasets", "truecar_test_data.csv")
truecar_prod_file = os.path.join(".", "TrimmedDatasets", "truecar_prod_data.csv")
craigslist_test_file = os.path.join(".", "TrimmedDatasets", "craigslist_test_data.csv")
craigslist_prod_file = os.path.join(".", "TrimmedDatasets", "craigslist_prod_data.csv")

# Declare our sizes for Test and Production dataframe
test_dataframe_size = 50
prod_dataframe_size = 250


# Create TrueCar Test and Production dataframes
truecar_random_index = np.random.choice(truecar_df.shape[0], replace=False, size=test_dataframe_size)
truecar_test_df = truecar_df.iloc[truecar_random_index]

truecar_random_index = np.random.choice(truecar_df.shape[0], replace=False, size=prod_dataframe_size)
truecar_prod_df = truecar_df.iloc[truecar_random_index]


# Create Craigslist Test and Production dataframes
craigslist_random_index = np.random.choice(craigslist_df.shape[0], replace=False, size=test_dataframe_size)
craigslist_test_df = craigslist_df.iloc[craigslist_random_index]

craigslist_random_index = np.random.choice(craigslist_df.shape[0], replace=False, size=prod_dataframe_size)
craigslist_prod_df = craigslist_df.iloc[craigslist_random_index]


# Output dataframes as CSVs
truecar_test_df.to_csv(truecar_test_file, header=True)
truecar_prod_df.to_csv(truecar_prod_file, header=True)
craigslist_test_df.to_csv(craigslist_test_file, header=True)
craigslist_prod_df.to_csv(craigslist_prod_file, header=True)

In [14]:
# Create merged Test and Production dataframes
# Create CSV output for combined dataframes

# Declare output files
comb_test_file = os.path.join(".", "TrimmedDatasets", "combined_test_data.csv")
comb_prod_file = os.path.join(".", "TrimmedDatasets", "combined_prod_data.csv")

# Perform concatentations
comb_test_df = pd.concat([truecar_test_df, craigslist_test_df], ignore_index=True)
comb_prod_df = pd.concat([truecar_prod_df, craigslist_prod_df], ignore_index=True)

# Output dataframes as CSVs
comb_test_df.to_csv(comb_test_file, header=True)
comb_prod_df.to_csv(comb_prod_file, header=True)


### NHTSA.GOV API Call - Obtain Clean Make and Model
Note: Due to the volume of cars in our database, we will be limiting tasks that involve API calls
to a subset of our dataset

In [17]:
# Use NHTSA VIN Decoder API to get a clean Make and Model for the Recall API calls 
# NOTE: VIN DECODER ONLY WORKS FOR YEARS 1981 AND NEWER
#       FOR INVALID VINs, THE DECODER WILL HAVE ["Results"][0]["ErrorCode"] != 0
#       FOR VALID VINs, THE DECODER WILL HAVE ["Results"][0]["ErrorCode"] = 0

vin_base_url = "https://vpic.nhtsa.dot.gov/api/vehicles/decodevinvalues/"
earliest_valid_year = 1981

# Create column to indicate whether VIN has been verified or not
comb_test_df["Vin_Verified"] = False

# Iterate through dataframe
for index, row in comb_test_df.iterrows(): 
    if row["Year"] >= earliest_valid_year:
        # Form query URL for current Vin and Year
        current_vin = row["Vin"]
        current_year = row["Year"]
        vin_query_url = f"{vin_base_url}{current_vin}?format=json&modelyear={current_year}"

        # Make API call
        response = requests.get(vin_query_url).json()

        # Check response object
        response_code = response["Results"][0]["ErrorCode"]
        if (response_code == "0"):
            # Successful response
            # Set Vin Verified to True
            comb_test_df.loc[index, "Vin_Verified"] = True

            # Update Make and Model from API call results
            comb_test_df.loc[index, "Make"]  = response["Results"][0]["Make"]
            comb_test_df.loc[index, "Model"] = response["Results"][0]["Model"]


### NHTSA.GOV API Call - Obtain Recall Data
Note: Due to the volume of cars in our database, we will be limiting tasks that involve API calls
to a subset of our dataset

In [ ]:
try:
  print(x)
except NameError:
  print("Variable x is not defined")
except:
  print("Something else went wrong")

In [39]:
# Use NHTSA Recall API to get recall information for each Vin Verified row of our dataframe

recall_base_url = f"https://one.nhtsa.gov/webapi/api/Recalls/vehicle/"

# Iterate through dataframe
for index, row in comb_test_df.iterrows(): 
    if row["Vin_Verified"] == True:
        # Only make Recall API calls for Vin Verified rows
        
        # Form query URL for current Year, Make, and Model
        current_year = row["Year"]
        current_make = row["Make"]
        current_model = row["Model"]
        current_vin = row["Vin"]

        recall_query_url = f"{recall_base_url}modelyear/{current_year}/make/{current_make}/model/{current_model}?format=json"

        try:
            # Make API call and store response
            response = requests.get(recall_query_url).json()

            if (response["Count"] > 0):
                # Recall data found
                current_recall_dict = {}
                current_recall_lst = []
                for x in range(response["Count"]):
                    current_recall_dict = {"NHTSACampaignNumber" : response["Results"][x]["NHTSACampaignNumber"]}

                    current_recall_lst.append(current_recall_dict)

                print(f"Recall data for: {current_vin} {current_year} {current_make} {current_model}:")
                print(current_recall_lst)
                print("-------------------------------------------------------")
        except json.decoder.JSONDecodeError:
            print("-------------------------------------------------------")
            print(f"JSON Decode error occurred for: {current_vin} {current_year} {current_make} {current_model}:")
            print("-------------------------------------------------------")
        except:
            print("-------------------------------------------------------")
            print(f"Other error occurred for: {current_vin} {current_year} {current_make} {current_model}:")
            print("-------------------------------------------------------")

        

Recall data for: 2FMHK6DT0ABA38716 2010 FORD Flex:
[{'NHTSACampaignNumber': '16V247000'}]
-------------------------------------------------------
Recall data for: 1GBFG15R211220159 2001 CHEVROLET Express:
[{'NHTSACampaignNumber': '08V649000'}, {'NHTSACampaignNumber': '08V650000'}]
-------------------------------------------------------
Recall data for: WVGWV7AX2HK016499 2017 VOLKSWAGEN Tiguan:
[{'NHTSACampaignNumber': '16V955000'}, {'NHTSACampaignNumber': '18V102000'}, {'NHTSACampaignNumber': '18V329000'}]
-------------------------------------------------------
Recall data for: 1N6AA1F23HN503727 2017 NISSAN Titan:
[{'NHTSACampaignNumber': '17V268000'}, {'NHTSACampaignNumber': '18V240000'}, {'NHTSACampaignNumber': '19V495000'}]
-------------------------------------------------------
Recall data for: 1G6AA5RA9G0197693 2016 CADILLAC ATS:
[{'NHTSACampaignNumber': '15V558000'}, {'NHTSACampaignNumber': '15V463000'}]
-------------------------------------------------------
Recall data for: 1N4